# 1.1 data malipuleren

In [1]:
pip install python-mnist -user


Usage:   
  /opt/jupyterhub/anaconda/bin/python -m pip install [options] <requirement specifier> [package-index-options] ...
  /opt/jupyterhub/anaconda/bin/python -m pip install [options] -r <requirements file> [package-index-options] ...
  /opt/jupyterhub/anaconda/bin/python -m pip install [options] [-e] <vcs project url> ...
  /opt/jupyterhub/anaconda/bin/python -m pip install [options] [-e] <local project path> ...
  /opt/jupyterhub/anaconda/bin/python -m pip install [options] <archive url/path> ...

no such option: -u
Note: you may need to restart the kernel to use updated packages.


In [2]:
import math
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as numpy
import pandas as pd


#import data
data=pd.read_csv('/datc/parcel/notebooks/data/postnl/20201014_300_klanten.csv', parse_dates=[0])

#klant 153 eruit halen
cust_filter = data['cust_id'] == 'klant_153'
data = data.where(cust_filter).dropna()
data

,procesdag,cust_id,aantal_pakketten,aantal_pakketten_volgende_dag,validation_column
47,2015-01-02,klant_153,5019.0,0.0,train
307,2015-01-03,klant_153,0.0,2713.0,train
567,2015-01-04,klant_153,2713.0,7699.0,train
827,2015-01-05,klant_153,7699.0,7341.0,train
1087,2015-01-06,klant_153,7341.0,4414.0,train
...,...,...,...,...,...
504447,2020-04-25,klant_153,0.0,1311.0,test
504707,2020-04-26,klant_153,1311.0,2.0,test
504967,2020-04-27,klant_153,2.0,8256.0,test
505227,2020-04-28,klant_153,8256.0,4951.0,test


# 1.2 train en test set

In [3]:
# Define amount of packages yesterday
data['aantal_pakketten_gisteren'] = data['aantal_pakketten'].shift(1)
data['aantal_pakketten_gisteren'].fillna(0, inplace=True)

In [7]:
#bron: https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/feedforward_neural_network/main.py#L37-L49

# Device configuration
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001


# Set conditions
data = data.where(data['procesdag'].dt.year < 2020).dropna()
test_size = 365 # 3 months
validation_size = 365 # 3 months

# Split train, test and validation
df_train = data[:-test_size]
df_test = data[-test_size:]
df_valid = data[-validation_size:]

# MNIST dataset 
#train_dataset = torchvision.datasets.MNIST(root=data, 
                                          # train=True, 
                                          # transform=transforms.ToTensor(),  
                                          # download=True)

#test_dataset = torchvision.datasets.MNIST(root=data, 
                                          #train=False, 
                                          #transform=transforms.ToTensor())

# Data loader
#train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           #batch_size=batch_size, 
                                           #shuffle=True)

#test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          #batch_size=batch_size, 
                                          #shuffle=False)



# 1.3 model maken

In [8]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
#model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# 1.4 loss

In [9]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

NameError: name 'model' is not defined

# 1.5 training

In [34]:

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        labels = labels.to(device)
        
        # Forward pass
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
        

AttributeError: 'str' object has no attribute 'to'

# 1.6 testing

In [35]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

ValueError: too many values to unpack (expected 2)